# Desafio Passei Direto - Data Engineer (Parte II)

In [2]:
# importando as bibliotecas necessárias
import findspark
findspark.init()

import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import substring

In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName('Passei Direto - Parte 2') \
    .master('local') \
    .getOrCreate()

# 1. Preparando os dados pare serem analisados

In [17]:
# read page views files 
session_data = os.path.join(os.getcwd(), "data/BASE_B/*.json")
df = spark.read.json(session_data)
df = df.toDF(*(c.lower() for c in df.columns))
print(df.count())
df.limit(5).toPandas().head()

698942


,last accessed url,page category,page category 1,page category 2,page category 3,page name,at,browser,carrier,city_name,...,name,nth,os_ver,platform,region,session_uuid,studentid_clienttype,type,user_type,uuid
0,/,perfil,perfil,Undefined,Undefined,/perfil/22482764/materiais,2017-11-16 02:10:20,Chrome 62,Telemar Norte Leste S.a.,None,...,Page View,17,,Linux,None,188031bec37fc43b737c2c49349076700ae89128,34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364...,e,known,1b3ed1360694ceae79f6361ed11b03cf245311c8
1,/disciplina/estudos-dirigidos-viii?type=6&mate...,busca,busca,Undefined,Undefined,/busca?disciplina=686342&disciplina=669865&dis...,2017-11-16 02:10:26,Firefox 56,Telefonica Brasil S.a.,None,...,Page View,19,,Windows 7,None,e3fe75ffce700fef07fd93e66312117ba008f046,23fa8b361cc7ea61dea852d72eba9129f05981077e1f65...,e,known,b0110b5beac39737f52678ffdba6c565bb72d8e2
2,/ranking/curso,disciplina,disciplina,marketing-pessoal,Undefined,/disciplina/marketing-pessoal,2017-11-16 02:10:22,Chrome 61,Tim Celular S.a.,None,...,Page View,2,,Windows 7,None,e0841700c23ea9aaadf1104690e7d6e12a8a771a,14cc805c72b0eec6292916da59c9c45a8aa621a1138eb6...,e,known,bea05ad3a2fc2b621bcdb048eafa4e70f5344b17
3,None,home,None,None,None,/,2017-11-16 02:10:34,Mobile Safari 10,Internexa Brasil Operadora De Telecomunicacoes...,None,...,Page View,2,10.3.2,iOS,None,318775acadbb0c72430ac2eeb05782589fbd1cf7,98774aeb803eb08e08020309fe8472af79033fbae26ea4...,e,known,a8ee07ca847a3126007ee0f5ceec8d6fa0d92727
4,None,disciplina,disciplina,cinesiologia,Undefined,/disciplina/cinesiologia?type=6&materialid=175...,2017-11-16 02:10:53,Chrome 61,Hipernet Servico De Comunicacao Ltda Me,None,...,Page View,11,,Windows 8.1,None,91bf1fbddb681569c914a8bb05fa0e260d9d92bd,122ac90aee4fc73660620456712cfea98d7bd720a1fc71...,e,known,ad6d53dc93ec8d8c30050f4391c9ecdf7da13710


In [20]:
# Removing @Website from the end of the string
df = df.withColumn("student_id", substring(df.studentid_clienttype, 0, 64))
df.select("student_id","studentid_clienttype").limit(1).collect()

In [22]:
# Join BASE_B with students.json
students = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/students.json"))
students = students.toDF(*(c.lower() for c in students.columns))
df_students = df.join(students, df.student_id == students.id, how='left')
print(df_students.count())
df_students.limit(5).toPandas().head()

698942


,last accessed url,page category,page category 1,page category 2,page category 3,page name,at,browser,carrier,city_name,...,uuid,student_id,city,courseid,id,registereddate,signupsource,state,studentclient,universityid
0,None,disciplina,disciplina,ciencia-politica,Undefined,/disciplina/ciencia-politica,2017-11-16 18:45:36,Chrome 61,Claro Brazil,None,...,2585063e8fa99055941f76ee937d928b8dc9c52f,9890d41d77e4474e1db0f3604d3aae8cf3b965600e94bc...,None,NaN,None,None,None,None,None,NaN
1,/explorar-materiais,busca,busca,Undefined,Undefined,/busca?q=desafio%20profissional,2017-11-16 18:46:06,Chrome 62,Telefonica Brasil S.a.,None,...,81c749e3fbb0617f41e346a7756853fedd673c30,a5d0f83e79388edc0d0a267d0556fb85ca00b1076cfabd...,None,NaN,None,None,None,None,None,NaN
2,None,disciplina,disciplina,microprocessadores-e-microcontroladores,Undefined,/disciplina/microprocessadores-e-microcontrola...,2017-11-16 17:48:21,Chrome 61,Claro Brazil,None,...,5e6f28fe28f163269e99b4ab6f31bf8b87dd4c22,73643df6867dc8deb30cbbc0e7f2090d9b368908c27e2f...,None,NaN,None,None,None,None,None,NaN
3,None,disciplina,disciplina,desenvolvimento-do-jovem-e-adolecente,Undefined,/disciplina/desenvolvimento-do-jovem-e-adolece...,2017-11-16 01:15:31,Chrome 61,Brasil Telecom Sa - Filial Distrito Federal,None,...,6cb0a610634dc216ca8db9aa1ebeeb35193be7d8,fdc2909eb5cdd2c1bb196d7111307ce5d1e7a2d683dd6e...,None,NaN,None,None,None,None,None,NaN
4,None,disciplina,disciplina,direito-penal-iii,Undefined,/disciplina/direito-penal-iii?type=6&materiali...,2017-11-16 01:15:16,Chrome 57,Telemar Norte Leste S.a.,None,...,192a4970dee45bc30cde44cb04c287baeed679ad,8540499a34fee9e3a373acd3907de96d012c557031a694...,None,NaN,None,None,None,None,None,NaN


In [30]:
# Join df_students with students.json
subscriptions = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/subscriptions.json"))
subscriptions = subscriptions.toDF(*(c.lower() for c in subscriptions.columns))
df_students_subscriptions = df_students.join(subscriptions, df_students.student_id == subscriptions.studentid, how='left')
print(df_students_subscriptions.count())
df_students_subscriptions.limit(5).toPandas().head()

698943


,last accessed url,page category,page category 1,page category 2,page category 3,page name,at,browser,carrier,city_name,...,courseid,id,registereddate,signupsource,state,studentclient,universityid,paymentdate,plantype,studentid
0,None,disciplina,disciplina,ciencia-politica,Undefined,/disciplina/ciencia-politica,2017-11-16 18:45:36,Chrome 61,Claro Brazil,None,...,NaN,None,None,None,None,None,NaN,None,None,None
1,/explorar-materiais,busca,busca,Undefined,Undefined,/busca?q=desafio%20profissional,2017-11-16 18:46:06,Chrome 62,Telefonica Brasil S.a.,None,...,NaN,None,None,None,None,None,NaN,None,None,None
2,None,disciplina,disciplina,microprocessadores-e-microcontroladores,Undefined,/disciplina/microprocessadores-e-microcontrola...,2017-11-16 17:48:21,Chrome 61,Claro Brazil,None,...,NaN,None,None,None,None,None,NaN,None,None,None
3,None,disciplina,disciplina,desenvolvimento-do-jovem-e-adolecente,Undefined,/disciplina/desenvolvimento-do-jovem-e-adolece...,2017-11-16 01:15:31,Chrome 61,Brasil Telecom Sa - Filial Distrito Federal,None,...,NaN,None,None,None,None,None,NaN,None,None,None
4,None,disciplina,disciplina,direito-penal-iii,Undefined,/disciplina/direito-penal-iii?type=6&materiali...,2017-11-16 01:15:16,Chrome 57,Telemar Norte Leste S.a.,None,...,NaN,None,None,None,None,None,NaN,None,None,None


In [31]:
# Filtering premium users
df_students_premium = df_students_subscriptions.filter(df_students_subscriptions.plantype.isNotNull())
print(df_students_premium.count())
df_students_premium.limit(5).toPandas().head()

2536


,last accessed url,page category,page category 1,page category 2,page category 3,page name,at,browser,carrier,city_name,...,courseid,id,registereddate,signupsource,state,studentclient,universityid,paymentdate,plantype,studentid
0,/listas,lista,lista,Undefined,Undefined,/lista/35952306/logica-de-programacao,2017-11-16 14:55:09,Chrome 62,Gigalink De Nova Friburgo Solucoes Em Rede Mul...,None,...,1199553,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...,2014-08-28 13:39:51.163000,Email,Rio de Janeiro,Website,664623,2017-11-09 11:25:51.023041,Mensal,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...
1,/disciplina/logica-de-programacao?type=6&mater...,listas,listas,Undefined,Undefined,/listas,2017-11-16 14:55:04,Chrome 62,Gigalink De Nova Friburgo Solucoes Em Rede Mul...,None,...,1199553,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...,2014-08-28 13:39:51.163000,Email,Rio de Janeiro,Website,664623,2017-11-09 11:25:51.023041,Mensal,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...
2,None,disciplina,disciplina,logica-de-programacao,Undefined,/disciplina/logica-de-programacao?type=6&mater...,2017-11-16 14:00:48,Chrome 62,Gigalink De Nova Friburgo Solucoes Em Rede Mul...,None,...,1199553,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...,2014-08-28 13:39:51.163000,Email,Rio de Janeiro,Website,664623,2017-11-09 11:25:51.023041,Mensal,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...
3,None,disciplina,disciplina,logica-de-programacao,Undefined,/disciplina/logica-de-programacao,2017-11-16 16:22:14,Chrome 62,Gigalink De Nova Friburgo Solucoes Em Rede Mul...,None,...,1199553,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...,2014-08-28 13:39:51.163000,Email,Rio de Janeiro,Website,664623,2017-11-09 11:25:51.023041,Mensal,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...
4,None,disciplina,disciplina,logica-de-programacao,Undefined,/disciplina/logica-de-programacao?type=6&mater...,2017-11-16 16:24:49,Chrome 62,Gigalink De Nova Friburgo Solucoes Em Rede Mul...,None,...,1199553,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...,2014-08-28 13:39:51.163000,Email,Rio de Janeiro,Website,664623,2017-11-09 11:25:51.023041,Mensal,292fb7c6f479e5b65304fdabea392b9c8722a2fc58a77f...


# 2. Estados em que os assinantes da passei direto acessaram a plataforma

In [37]:
df_students_premium.groupby('state') \
    .count() \
    .filter("`count` >= 10") \
    .sort(col("count").desc()) \
    .collect()

[Row(state='Rio de Janeiro', count=686),
 Row(state='Pernambuco', count=376),
 Row(state='São Paulo', count=319),
 Row(state=None, count=306),
 Row(state='Ceará', count=177),
 Row(state='Santa Catarina', count=122),
 Row(state='Minas Gerais', count=120),
 Row(state='Rio Grande do Sul', count=113),
 Row(state='Paraná', count=70),
 Row(state='Bahia', count=55),
 Row(state='Goiás', count=53),
 Row(state='Maranhão', count=42),
 Row(state='Espírito Santo', count=29),
 Row(state='Sergipe', count=27),
 Row(state='Alagoas', count=12),
 Row(state='Distrito Federal', count=11)]

# 3. Cidades em que os assinantes da passei direto acessaram a plataforma

In [36]:
df_students_premium.groupby('city') \
    .count() \
    .filter("`count` >= 10") \
    .sort(col("count").desc()) \
    .collect()

[Row(city=None, count=1599),
 Row(city='Rio de Janeiro', count=248),
 Row(city='Recife', count=63),
 Row(city='Embu', count=61),
 Row(city='Fortaleza', count=50),
 Row(city='Niterói', count=42),
 Row(city='Lages', count=41),
 Row(city='Rio Pardo', count=31),
 Row(city='Nova Iguacu', count=29),
 Row(city='Aracaju', count=26),
 Row(city='Curitiba', count=25),
 Row(city='Florianópolis', count=25),
 Row(city='Barueri', count=24),
 Row(city='Goiânia', count=22),
 Row(city='Porto Alegre', count=22),
 Row(city='São Paulo', count=16),
 Row(city='Belo Horizonte', count=15),
 Row(city='Vitória', count=15),
 Row(city='Gravataí', count=14),
 Row(city='Salvador', count=14),
 Row(city='Itamaraju', count=14),
 Row(city='Palhoca', count=13),
 Row(city='São José', count=12),
 Row(city='Mesquita', count=12),
 Row(city='Juiz de Fora', count=11),
 Row(city='Euclides Da Cunha', count=11),
 Row(city='Brasília', count=10)]

# 4. Formas de inscrição mais comuns entre os assinantes da Passei Direto

In [38]:
df_students_premium.groupby('signupsource') \
    .count() \
    .filter("`count` >= 100") \
    .sort(col("count").desc()) \
    .collect()

[Row(signupsource='Facebook', count=1003),
 Row(signupsource='Google', count=808),
 Row(signupsource='Email', count=725)]

# 5. Categorias de páginas acessadas mais comuns entre os assinantes da Passei Direto

In [46]:
df_students_premium.groupby('page category') \
    .count() \
    .filter("`count` >= 10") \
    .sort(col("count").desc()) \
    .collect()

[Row(page category='disciplina', count=1743),
 Row(page category='home', count=221),
 Row(page category='busca', count=133),
 Row(page category='arquivo', count=124),
 Row(page category='pagamento', count=69),
 Row(page category='premium', count=48),
 Row(page category='listas', count=42),
 Row(page category='perfil', count=40),
 Row(page category='lista', count=35),
 Row(page category='configuracoes', count=29),
 Row(page category='cadastro', count=24),
 Row(page category='meu-premium', count=13),
 Row(page category='explorar-materiais', count=12)]

In [47]:
df_students_premium.groupby('page category 1') \
    .count() \
    .filter("`count` >= 10") \
    .sort(col("count").desc()) \
    .collect()

[Row(page category 1='disciplina', count=1737),
 Row(page category 1='Undefined', count=206),
 Row(page category 1=None, count=191),
 Row(page category 1='busca', count=120),
 Row(page category 1='pagamento', count=69),
 Row(page category 1='listas', count=42),
 Row(page category 1='perfil', count=38),
 Row(page category 1='lista', count=35),
 Row(page category 1='configuracoes', count=29),
 Row(page category 1='cadastro', count=23),
 Row(page category 1='arquivo', count=19),
 Row(page category 1='meu-premium', count=13),
 Row(page category 1='explorar-materiais', count=12)]

In [48]:
df_students_premium.groupby('page category 2') \
    .count() \
    .filter("`count` >= 10") \
    .sort(col("count").desc()) \
    .collect()

[Row(page category 2='Undefined', count=608),
 Row(page category 2=None, count=191),
 Row(page category 2='psicologia-institucional', count=110),
 Row(page category 2='matematica-financeira', count=48),
 Row(page category 2='metodologia-cientifica', count=45),
 Row(page category 2='pedagogia-nas-instituicoes-nao-escolares', count=44),
 Row(page category 2='bromatologia', count=41),
 Row(page category 2='psicologia-nas-organizacoes', count=38),
 Row(page category 2='gestao-da-qualidade', count=35),
 Row(page category 2='teoria-geral-da-administracao', count=31),
 Row(page category 2='matematica-para-negocios', count=30),
 Row(page category 2='contabilidade-social', count=28),
 Row(page category 2='mercado-financeiro', count=28),
 Row(page category 2='logica-de-programacao-e-algoritmos', count=24),
 Row(page category 2='analise-organizacional', count=23),
 Row(page category 2='direito-penal-iii', count=22),
 Row(page category 2='ensino-clinico-iii---teorico', count=21),
 Row(page categor

In [50]:
df_students_premium.groupby('page category 3') \
    .count() \
    .sort(col("count").desc()) \
    .collect()

[Row(page category 3='Undefined', count=2343),
 Row(page category 3=None, count=191),
 Row(page category 3='exercicios-resolvidos', count=2)]

# 6. Cursos mais populares dos assinantes que acessarram a plataforma

In [51]:
courses = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/courses.json"))
courses = courses.toDF(*(c.lower() for c in courses.columns))
courses = courses.withColumnRenamed("name", "course_name")
df_students_premium_courses = df_students_premium.join(courses, df_students_premium.courseid  == courses.id, how='left')

In [52]:
df_students_premium_courses.groupby('course_name') \
    .count() \
    .filter("`count` >= 100") \
    .sort(col("count").desc()) \
    .collect()

[Row(course_name='Direito', count=452),
 Row(course_name='Pedagogia', count=386),
 Row(course_name='Administração', count=292),
 Row(course_name='Engenharia de Produção', count=157),
 Row(course_name='Nutrição', count=127),
 Row(course_name='Contabilidade / Ciências Contábeis', count=112)]

# 7. Universidades mais populares dos assinantes que acessaram a plataforma

In [53]:
universities = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/universities.json"))
universities = universities.toDF(*(c.lower() for c in universities.columns))
universities = universities.withColumnRenamed("name", "university_name")
df_students_premium_universities = df_students_premium.join(universities, df_students_premium.universityid  == universities.id, how='left')

In [54]:
df_students_premium_universities.groupby('university_name') \
    .count() \
    .sort(col("count").desc()) \
    .collect()

[Row(university_name='ESTÁCIO', count=1622),
 Row(university_name='UNINTER', count=276),
 Row(university_name='ESTÁCIO EAD', count=194),
 Row(university_name='UNIP', count=110),
 Row(university_name='UFSC', count=41),
 Row(university_name='ANHANGUERA', count=28),
 Row(university_name='UFS', count=21),
 Row(university_name='UFC', count=21),
 Row(university_name='UNIASSELVI', count=21),
 Row(university_name='FAL', count=17),
 Row(university_name='IPESU', count=16),
 Row(university_name='UFBA', count=14),
 Row(university_name='UNISEB', count=12),
 Row(university_name='RADIAL', count=11),
 Row(university_name='UNIDERP - ANHANGUERA', count=11),
 Row(university_name='USP-SP', count=11),
 Row(university_name='UNINASSAU MACEIÓ', count=11),
 Row(university_name='UniAGES', count=11),
 Row(university_name='UFES', count=10),
 Row(university_name='UNICEUB', count=8),
 Row(university_name='UNIT', count=8),
 Row(university_name='UNDB', count=7),
 Row(university_name='UNICSUL', count=7),
 Row(universi

# Disciplinas mais populares dos assinantes que acessaram a plataforma

In [56]:
student_follow_subject = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/student_follow_subject.json"))
student_follow_subject = student_follow_subject.toDF(*(c.lower() for c in student_follow_subject.columns))
df_students_premium_student_follow_subject = df_students_premium.join(student_follow_subject, df_students_premium.id  == student_follow_subject.studentid, how='left')

In [58]:
subjects = spark.read.json(os.path.join(os.getcwd(), "data/BASE_A/subjects.json"))
subjects = subjects.toDF(*(c.lower() for c in subjects.columns))
subjects = subjects.withColumnRenamed("name", "subject_name")
df_students_premium_student_subjects = df_students_premium_student_follow_subject.join(subjects, df_students_premium_student_follow_subject.subjectid  == subjects.id, how='left')

In [61]:
df_students_premium_student_subjects.groupby('subject_name') \
    .count() \
    .filter("`count` >= 100") \
    .sort(col("count").desc()) \
    .collect()

[Row(subject_name='Educação Profissional: Teoria e Prática', count=261),
 Row(subject_name='Sustentabilidade', count=251),
 Row(subject_name='Estágio Supervisionado Fundamental', count=249),
 Row(subject_name='Supervisão e Orientação Pedagógica', count=249),
 Row(subject_name='Direito Constitucional I', count=170),
 Row(subject_name='Matemática Financeira', count=163),
 Row(subject_name='Orçamento Empresarial', count=147),
 Row(subject_name='Planejamento de Carreira e Sucesso Profissional', count=140),
 Row(subject_name='Teoria Geral do Processo', count=137),
 Row(subject_name='Direito Civil I', count=134),
 Row(subject_name='Contabilidade Básica', count=128),
 Row(subject_name='Direito Constitucional II', count=127),
 Row(subject_name='Mercado Financeiro', count=118),
 Row(subject_name='Administração de Marketing', count=117),
 Row(subject_name='Matemática para Negócios', count=116),
 Row(subject_name='Bioquímica', count=113),
 Row(subject_name='Direito Civil III', count=112),
 Row(su